# Lesson 18: 저출산 자료 시각화 (지저분한 데이터)

최규빈  
2023-07-26

<a href="https://colab.research.google.com/github/guebin/PP2023SUM/blob/main/posts/Day3/08_PandasBackend/ls18.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 강의영상

<https://youtu.be/F3n1jCM3PEI>

# imports

In [1]:
import pandas as pd 
import numpy as np

# 자료분석: 저출산

## 데이터읽기 // `pd.read_html()`

`-` 대한민국의 저출산문제

-   ref: https://ko.wikipedia.org/wiki/대한민국의_저출산

`-` 위의 url에서 3,5번째 테이블을 읽고싶다. - 3번째 테이블: 시도별
출산율 - 5번째 테이블: 시도별 출생아 수

In [2]:
_dflst = pd.read_html('https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%A0%80%EC%B6%9C%EC%82%B0')
_df1 = _dflst[2] 
_df2 = _dflst[4] 

In [3]:
pd.DataFrame(np.array([[1,2],[3,4]])).applymap(lambda x: x+1)

In [4]:
_df2

`-` 데이터정리

In [5]:
df1 = _df1.drop(17)\
.melt(id_vars='지역/연도[6]')\
.assign(variable = lambda df: list(map(lambda x: x[:4], df.variable)))\
.assign(value = lambda df: list(map(lambda x: None if x=='-' else float(x), df.value)))\
.set_axis(['지역','연도','출산율'],axis=1)
df1

In [6]:
df2 = _df2.drop(17)\
.melt(id_vars='지역/연도[6]')\
.assign(value = lambda df: list(map(lambda x: None if x=='-' else float(x), df.value)))\
.set_axis(['지역','연도','출생아수'],axis=1)
df2

## 시각화I: 전국 출생아수 시각화

In [7]:
df2.groupby(['연도'])\
.agg({'출생아수':np.sum})\
.reset_index()\
.plot(x='연도',y='출생아수',backend='plotly')

-   일괄적으로 감소하는 느낌은 없음

## 시각화II: 시도별 출생아수 시각화

`-` 시각화예시1

In [8]:
df2.plot.line(backend='plotly', x='연도',y='출생아수',color='지역')

-   서울과 경기가 특이함

`-` 시각화예시2

In [9]:
df2.plot.area(backend='plotly',x='연도',y='출생아수',color='지역')

-   areaplot의 최상단의 선: 전국출생아수 시각화와 같음 (일괄적으로
    감소하는 느낌은 별로 없음, 그 이유를 살펴보니 서울과 경기지역
    떄문임)
-   areaplot의 장점: 전국출생아수를 년도별로 시각화 하는 느낌 + 각
    연도를 도시별로 분해하여 해석하는 느낌

## 시각화III: 시도별 출산율 시각화

In [10]:
df1.plot.line(backend='plotly',x='연도',y='출산율',color='지역')

-   상식과 일치하는 정상적인 플랏 (출산율이 2012년 이후로 꺽이는 느낌이
    든다)
-   여기서는 서울/경기가 정상인듯 보인다.

> 출산율의 경우 합계출산율이 크게 의미가 없으므로 areaplot은 생략한다.

## 해석

`-` 이상한점: 서울/경기지역에서 특정연도의 출생아수가 매우 낮음. 그런데
서울/경기지역의 출산율은 모든 년도에서 고른값을 가짐.

`-` 해석: 데이터가 이상하다.. // 위키를 살펴보니 오타가 있음!!

## 데이터의 수정 (1): df2 상태에서 수정

`-` df2의 수정

In [15]:
df2.sort_values("출생아수")[:10]

`-` 오타로 예상되는 서울/경기/충남 이외의 가장 작은 값은 2012년
세종시인데, 이 값이 1054로 1000보다 크다.

-   출생아수 \< 1000 이면 출생아수 \* 1000 을 수행하는 함수를 구현하자.

In [17]:
df2.assign(출생아수= df2.출생아수.apply(lambda x: x*1000 if x<1000 else x))

`-` 잘 변환되었는지 확인하기 위한 시각화

In [18]:
df2.assign(출생아수= df2.출생아수.apply(lambda x: x*1000 if x<1000 else x))\
.plot.area(x='연도',y='출생아수',color='지역',backend='plotly')

-   상식적인 결과: 전체출산율이 점점 낮아지고 있고 항목별로 살펴보아도
    모든 도시의 출생아수가 점차 낮아지고 있음

## 데이터의 수정 (2): \_df2 상태에서 수정

`-` applymap

In [19]:
_df2.set_index('지역/연도[6]') # applymap을 쓰기 위해서 임시로 지역/연도[6]을 인덱스로만듬 

`-` 방법1

In [51]:
_df2.set_index('지역/연도[6]')\
.applymap(lambda x: None if x == '-' else float(x))\
.applymap(lambda x: x*1000 if x<1000 else x)\
.drop('전국')\
.stack().reset_index()

`-` 방법2

In [21]:
df2 = _df2.set_index('지역/연도[6]')\
.applymap(lambda x: None if x == '-' else float(x))\
.applymap(lambda x: x*1000 if x<1000 else x)\
.drop('전국')\
.reset_index()\
.melt(id_vars='지역/연도[6]')\
.set_axis(['지역','연도','출생아수'],axis=1)
df2

In [22]:
df2.plot.area(backend='plotly',x='연도',y='출생아수',color='지역')